In [28]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import os
import cv2 as cv
import pickle
import random
from skimage import feature
import pandas as pd
from pathlib import Path
from sklearn.svm import LinearSVC

In [ ]:
print("Enter datapath for training dataset")
DATADIR = Path(str(input()))
print("The path for training dataset is: ",DATADIR)

In [30]:
Categories = ['Crazing','Pitted','Inclusion','Patches','Rolled','Scratches']

In [31]:
class LocalBinaryPatterns:
    # store the number of points and radius
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius


    def describe (self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
                                       self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
             bins=np.arange(0, self.numPoints + 3),
             range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist
                                 
# compute the Local Binary Pattern representation of the image, and 
# use the LBP representation to build the histogram of patterns

In [32]:
desc = LocalBinaryPatterns(24, 8)
data = []

In [33]:
for category in Categories:
    path = os.path.join(DATADIR, category) # path to respective folders
    label = Categories.index(category)
    for img in os.listdir(path):
        img_array = cv.imread(os.path.join(path,img),cv.COLOR_BGR2GRAY)
        hist = desc.describe(img_array)
        data.append([hist,label])

In [34]:
#pick_in = open('data_LBP+SVM.pickle','wb')
#pickle.dump(data,pick_in)
#pick_in.close()

In [35]:
#pick_in = open('data_LBP+SVM.pickle','rb')
#data = pickle.load(pick_in)
#pick_in.close()

In [36]:
random.shuffle(data)

In [37]:
features = []
labels =[]

In [38]:
for hist,label in data:
    features.append(hist)
    labels.append(label)

In [40]:
model = LinearSVC(C=100.0, random_state=42, max_iter=10000)
model.fit(features, labels)

LinearSVC(C=100.0, max_iter=10000, random_state=42)

In [58]:
### for saving the trained model##
# pick_LBP_SVM = open('model_LBP+SVM.sav','wb')
# pickle.dump(model,pick_LBP_SVM)
# pick_LBP_SVM.close()

In [43]:
### for importing the trained model##
# pick_LBP_SVM = open('model_LBP+SVM.sav','rb')
# model = pickle.load(pick_LBP_SVM)
# pick_LBP_SVM.close()

In [44]:
data_test= []
actual_image = []

In [ ]:
print("Enter datapath for test dataset")
DATADIR_test = Path(str(input()))
Categories_test = ['Crazing','Pitted','Inclusion','Patches','Rolled','Scratches']

In [46]:
for category in Categories_test:
    path_test = os.path.join(DATADIR_test, category) # path to respective folders
    label_test = Categories_test.index(category)
    for img_test in os.listdir(path_test):
        img_array = cv.imread(os.path.join(path_test,img_test),cv.COLOR_BGR2GRAY)
        hist = desc.describe(img_array)        
        data_test.append([hist,label_test,img_test])

In [47]:
features_test = []
labels_test = []

In [48]:
for hist_test,label_test,img_test in data_test:
    features_test.append(hist_test)
    labels_test.append(label_test)
    actual_image.append(img_test)

In [61]:
prediction = model.predict(features_test)

In [62]:
print(f'Accuracy: {accuracy_score(prediction,labels_test)*100}% accuracy')

Accuracy: 88.88888888888889% accuracy


In [51]:
print(classification_report(prediction,test_df.iloc[:,-1]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.92      0.96        13
           2       0.50      1.00      0.67         6
           3       1.00      0.92      0.96        13
           4       1.00      1.00      1.00        12
           5       0.83      0.62      0.71        16

    accuracy                           0.89        72
   macro avg       0.89      0.91      0.88        72
weighted avg       0.92      0.89      0.89        72

